In [1]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections
import functools

In [2]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])

In [3]:
possible_plans = [
        ['orange','black','green'],
        ['blue','black','yellow'],
        ['blue','green','orange'],
        ['yellow','green','black'],
        ['black','yellow','orange'],
        ['green','yellow','blue'],
        ['blue','orange','black'],
        ['green','orange','yellow'],
        ['black','blue','green'],
        ['orange','blue','yellow']
    ]

colormap = {
    'black'  : 0,
    'green'  : 1,
    'blue'   : 2,
    'orange' : 3,
    'yellow' : 4
}

In [6]:
"""
    places in heaps: (1 is closer to orange side, 0 to central tracking device)
       | 1|
    | 0| 4| 2|
       | 3|

    colors of cubes: (same as for orange side)
    
    black  -- 0
    green  -- 1
    blue   -- 2
    orange -- 3
    yellow -- 4
    
    numbers of heaps - 0,1,2,3,4,5 CCW starting from closest heap to central tracking device on the orange side
    
    man = 0, 1, 2 (picking 1 4 3 if from 0 for orange!)
    
    cubes:
    -1 - none
    
""";


In [7]:
class STNode():
    def __init__(self, state, upper):
        self.state = state
        self.upper = upper
        self.nexts = []
        
    def __str__(self):
        return str(self.state)
    
    def gen_reversed_path(self):
        sn = self
        while sn != None:
            yield sn
            sn = sn.upper
    
    def get_path(self):
        return reversed(list(self.gen_reversed_path()))

In [40]:
orange_heap = np.array([
                        [-1, 0,-1],
                        [ 1, 4, 3],
                        [-1, 2,-1]
                    ],dtype=np.float)
green_heap  = np.array([
                        [-1, 0,-1],
                        [ 3, 4, 1],
                        [-1, 2,-1]
                 ],dtype=np.float)
orange_heap[orange_heap == -1] = np.nan
green_heap[green_heap == -1] = np.nan
def get_heap(heap_num, side):
    heap = None
    if number in [0,1,2]:
        heap = orange_heap
    if number in [3,4,5]:
        heap = green_heap
    return np.rot90(heap,side)

In [42]:
class CubesState():
    initial_cubes = np.ones((3,5),dtype=np.int)*np.nan
    magic_color = 7
    def __init__(self, heap, cubes = None, **kvargs):
        self.heap = heap
        self.cubes = cubes if cubes else copy.deepcop(self.initial_cubes)
        self.vars = kvargs
        self.n_picks = 0
        self.time = 0
        self.x = 0
        self.y = 0
        self.a = 0
        self.params = kvargs
        self.set_defaults()
    
    def set_defaults(self):
        pass
    
    def remove_color(self, color):
        self.heap[self.heap == color] = np.nan
        
    def cubes_left(self):
        return np.sum(self.heap == self.heap)
    
    def pick_cube_by(self, color, man):
        self.cubes[man] = np.roll(self.cubes[man])
        self.cubes[man][0] = int(color)
    
    def has_free_space(self,man):
        return not np.all(self.cubes[man] == self.cubes[man])
    
    def pick_cubes(self,colors,mans):
        for c,m in zip(colors, mans):
            self.pick_cube_by(c,m)
    
    def test_pick_cubes(self, colors, mans):
        for c, m in zip(colors,mans):
            if not self.has_free_space(m) or c != c:
                return False
        return True
    
    def is_full(self):
        return np.all(self.cubes == self.cubes)
    
    def count_plans(self, plan):
        rev_plan = np.array(list(reversed(plan)))
        has_plan = np.zeros((3,), dtype=np.bool)
        for man, s in iterer(3,3):
            subs = self.cubes[man][s:s+3]
            if np.all(subs == plan) or np.all(subs == rev_plan):
                has_plan = True
        return np.sum(has_plan)
    
    def __str__(self):
        s = "---STATE---" + '\n'
        for l in self.cubes:
            s += str(l) + '\n'
        for k,v in self.vars.items():
            s += "%s : %s\t"%(str(k),str(v))
        for m in self.cubes:
            for c in m:
                if c == c:
                    s += "%d "%c
            s += '\n'
        s += '-----------'
        return s

In [ ]:
class MainRobotState(CubesState):
    defaults = {
            'move_time'  :1,
            'pick_time'  :1,
            'rotate_time':1
        }
    shift = np.array([[0,-1],[-1,0],[0,1],[1,0]])
    mans_shift = np.array([-1,0,1])
    def set_defaults(self):
        for k,v in self.defaults.items():
            if k not in self.params:
                self.params[k] = v
    
    def get_mans_x(self):
        x = np.array((3,2),dtype=np.int)
        x[:,0] = self.x + 1
        x[:,1] = self.y + 1
        a_index = (self.mans_shift + self.a) % 4
        return x + shift[a_index]
    
    def check_mans_x(self):
        R = np.zeros((3,),dtype=np.bool)
        X = self.get_mans_x()
        for r,x in zip(R,X):
            r = np.all(np.logical_and(x >= 0, x < 2)) and self.heap[x] == self.heap[x]
            
    def get_colors(self, mans):
        
        return self.heap[x]
    
    def get_time_between(self, s):
        return np.max(np.abs(self.x - s.x),np.abs(self.y - s.y))*self.params['move_time'] +
                np.min(np.abs(self.a - s.a), np.abs(self.a - s.a - 4), np.abs(self.a - s.a + 4))*self.params['rotate_time']
        
    def next_moves(self):
        if self.cubes_left() in [3,4,5]:
            for a in range(4):
                s = copy.deepcopy(self)
                s.a = a
                s.time += self.get_time_between(s)
                yield s
        else:
            
                

In [52]:
a = (np.array([-1,0,1]) + 3) % 4

In [61]:
np.logical_and(0 <= a, a < 3)

array([ True, False,  True])